# Comprehensive Generative AI Application
## Text, Image, and Code Generation using Free/Open-Source Models

This notebook demonstrates multiple generative AI capabilities using Hugging Face models and Ollama for local LLM execution.

## 1. Import Required Libraries

Install and import all necessary dependencies for generative AI tasks.

In [ ]:
import subprocess
import sys

# Install required packages
packages = [
    'transformers',
    'torch',
    'diffusers',
    'pillow',
    'requests',
    'accelerate',
    'safetensors'
]

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

In [ ]:
import torch
import numpy as np
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    GPT2Tokenizer,
    GPT2LMHeadModel
)
from diffusers import StableDiffusionPipeline
from PIL import Image
import requests
from io import BytesIO
import time
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Running on CPU (slower, but will work)")

## 2. Set Up Free/Open-Source Models

Configure models from Hugging Face and set up device management for optimal performance.

In [ ]:
# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Model configurations for different tasks
MODEL_CONFIGS = {
    "text_generation": {
        "gpt2": "gpt2",
        "distilgpt2": "distilgpt2",  # Smaller, faster version
    },
    "summarization": {
        "t5-small": "t5-small",
        "distilbart": "sshleifer/distilbart-cnn-6-6"
    },
    "question_answering": {
        "distilbert": "distilbert-base-cased-distilled-squad"
    },
    "image_generation": {
        "stable_diffusion": "runwayml/stable-diffusion-v1-5"
    }
}

# Initialize a dictionary to store loaded models
loaded_models = {}

def load_model_with_status(model_name, task_type, use_fp16=False):
    """Load a model and report status"""
    print(f"\n📥 Loading {task_type} model: {model_name}")
    start_time = time.time()
    
    try:
        if task_type == "text_generation":
            pipe = pipeline("text-generation", model=model_name, device=0 if device == "cuda" else -1)
        elif task_type == "summarization":
            pipe = pipeline("summarization", model=model_name, device=0 if device == "cuda" else -1)
        elif task_type == "question_answering":
            pipe = pipeline("question-answering", model=model_name, device=0 if device == "cuda" else -1)
        
        elapsed = time.time() - start_time
        print(f"✓ Successfully loaded in {elapsed:.2f} seconds")
        return pipe
    except Exception as e:
        print(f"✗ Error loading model: {e}")
        return None

# Load GPT-2 for text generation
gpt2_model = load_model_with_status("distilgpt2", "text_generation")

## 3. Text Generation with Hugging Face

Generate creative text using pre-trained transformer models with various parameters.

In [ ]:
def generate_text(prompt, model, max_length=100, temperature=0.7, top_p=0.9, num_outputs=1):
    """
    Generate text using a language model
    
    Parameters:
    - prompt: Starting text
    - model: Loaded pipeline model
    - max_length: Maximum length of generated text
    - temperature: Controls randomness (0=deterministic, 1=more random)
    - top_p: Nucleus sampling parameter
    - num_outputs: Number of outputs to generate
    """
    print(f"\n🎯 Generating text from prompt: '{prompt}'")
    print(f"Parameters: max_length={max_length}, temperature={temperature}, top_p={top_p}")
    
    try:
        outputs = model(
            prompt,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            num_return_sequences=num_outputs,
            do_sample=True
        )
        
        for i, output in enumerate(outputs):
            print(f"\n--- Generation {i+1} ---")
            print(output['generated_text'])
        
        return outputs
    except Exception as e:
        print(f"Error generating text: {e}")
        return None

# Example 1: Simple creative writing
print("=" * 60)
print("EXAMPLE 1: Creative Story Generation")
print("=" * 60)
generate_text(
    "Once upon a time in a magical forest",
    gpt2_model,
    max_length=80,
    temperature=0.9,
    num_outputs=1
)

# Example 2: Technical documentation
print("\n" + "=" * 60)
print("EXAMPLE 2: Technical Content Generation")
print("=" * 60)
generate_text(
    "Python decorators are useful for",
    gpt2_model,
    max_length=100,
    temperature=0.5,
    num_outputs=1
)

# Example 3: Multiple variations
print("\n" + "=" * 60)
print("EXAMPLE 3: Multiple Variations")
print("=" * 60)
generate_text(
    "The future of artificial intelligence",
    gpt2_model,
    max_length=80,
    temperature=0.8,
    num_outputs=2
)

## 4. Image Generation with Stable Diffusion

Generate images from text prompts using the Stable Diffusion model.

In [ ]:
print("\n" + "=" * 60)
print("Loading Stable Diffusion model...")
print("(This may take a while on first run)")
print("=" * 60)

try:
    # Load Stable Diffusion pipeline
    print("📥 Loading Stable Diffusion...")
    start_time = time.time()
    
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )
    pipe = pipe.to(device)
    
    elapsed = time.time() - start_time
    print(f"✓ Successfully loaded in {elapsed:.2f} seconds")
    
    stable_diffusion_loaded = True
except Exception as e:
    print(f"⚠ Note: Stable Diffusion requires significant resources")
    print(f"Error: {e}")
    print("We'll skip image generation for now")
    stable_diffusion_loaded = False

def generate_image(prompt, num_steps=50, guidance_scale=7.5):
    """
    Generate images from text prompts
    
    Parameters:
    - prompt: Text description of image
    - num_steps: Number of inference steps (more = better but slower)
    - guidance_scale: Controls adherence to prompt (higher = more adherent)
    """
    if not stable_diffusion_loaded:
        print("⚠ Stable Diffusion is not loaded. Skipping image generation.")
        return None
    
    print(f"\n🎨 Generating image from prompt: '{prompt}'")
    print(f"Parameters: num_steps={num_steps}, guidance_scale={guidance_scale}")
    
    try:
        start_time = time.time()
        
        image = pipe(
            prompt,
            num_inference_steps=num_steps,
            guidance_scale=guidance_scale
        ).images[0]
        
        elapsed = time.time() - start_time
        print(f"✓ Image generated in {elapsed:.2f} seconds")
        
        return image
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

# Example image generation (optional - only if model loaded successfully)
if stable_diffusion_loaded:
    print("\n" + "=" * 60)
    print("EXAMPLE: Image Generation")
    print("=" * 60)
    image = generate_image(
        "A serene mountain landscape with a lake at sunset, oil painting style",
        num_steps=30,
        guidance_scale=7.5
    )
    
    if image:
        image.save("generated_image.png")
        print("✓ Image saved as 'generated_image.png'")
else:
    print("\n" + "=" * 60)
    print("Image Generation - Resource Limitation")
    print("=" * 60)
    print("💡 Stable Diffusion requires 6-8GB VRAM for optimal performance")
    print("💡 For image generation, consider using an online API or local Ollama setup")
    print("💡 You can modify the model to a lighter version if available")

## 5. Code Generation with Open-Source Models

Generate Python code snippets from natural language descriptions.

In [ ]:
# Load CodeGen or similar model for code generation
print("\n" + "=" * 60)
print("Loading Code Generation Model")
print("=" * 60)

try:
    code_gen_model = load_model_with_status("Salesforce/codegen-350M-mono", "text_generation")
    code_gen_loaded = True
except:
    code_gen_loaded = False
    print("⚠ CodeGen model not available, using GPT-2 for code examples")
    code_gen_model = gpt2_model

def generate_code(description, model, max_length=150, temperature=0.5):
    """
    Generate Python code from a natural language description
    
    Parameters:
    - description: What the code should do
    - model: Loaded code generation model
    - max_length: Max tokens to generate
    - temperature: Creativity level
    """
    print(f"\n💻 Generating code for: '{description}'")
    
    # Create a prompt in the format the model understands
    prompt = f"# {description}\n"
    
    try:
        outputs = model(
            prompt,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=1,
            do_sample=True
        )
        
        generated_code = outputs[0]['generated_text']
        print("\n--- Generated Code ---")
        print(generated_code)
        
        return generated_code
    except Exception as e:
        print(f"Error: {e}")
        return None

# Code generation examples
print("\n" + "=" * 60)
print("EXAMPLE 1: Function Generation")
print("=" * 60)
generate_code(
    "Function to calculate factorial of a number",
    code_gen_model,
    max_length=120,
    temperature=0.5
)

print("\n" + "=" * 60)
print("EXAMPLE 2: Data Processing")
print("=" * 60)
generate_code(
    "Read CSV file and filter rows where age > 25",
    code_gen_model,
    max_length=150,
    temperature=0.6
)

print("\n" + "=" * 60)
print("EXAMPLE 3: Web Request Handler")
print("=" * 60)
generate_code(
    "Function to make HTTP GET request and handle errors",
    code_gen_model,
    max_length=150,
    temperature=0.5
)

## 6. Advanced Features: Summarization & Q&A

Bonus capabilities for text understanding and content creation.

In [ ]:
# Load summarization model
print("\n" + "=" * 60)
print("Loading Summarization Model")
print("=" * 60)

try:
    summarization_model = load_model_with_status("facebook/bart-large-cnn", "summarization")
    summarization_loaded = True
except:
    try:
        summarization_model = load_model_with_status("sshleifer/distilbart-cnn-6-6", "summarization")
        summarization_loaded = True
    except:
        summarization_loaded = False
        print("⚠ Summarization model not available")

def summarize_text(text, max_length=100, min_length=50):
    """
    Summarize a long text into a shorter version
    
    Parameters:
    - text: Text to summarize
    - max_length: Maximum length of summary
    - min_length: Minimum length of summary
    """
    if not summarization_loaded:
        print("⚠ Summarization model not loaded")
        return None
    
    print(f"\n📝 Summarizing text...")
    print(f"Original length: {len(text.split())} words")
    
    try:
        summary = summarization_model(
            text,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )
        
        summary_text = summary[0]['summary_text']
        print(f"Summary length: {len(summary_text.split())} words")
        print(f"\n--- Summary ---")
        print(summary_text)
        
        return summary_text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example text for summarization
sample_text = """
Artificial Intelligence has rapidly transformed from a niche research field to 
one of the most influential technologies of our time. Machine learning models, 
particularly deep neural networks, have achieved remarkable breakthroughs in 
various domains including computer vision, natural language processing, and 
game playing. Recent advances in transformer architectures have led to the 
development of large language models that can understand and generate human-like 
text at scale. These models are now being integrated into countless applications, 
from virtual assistants to content creation tools. However, alongside these 
impressive capabilities, important questions about AI safety, bias, and ethics 
have emerged. Researchers and organizations worldwide are working to ensure that 
AI systems are developed responsibly and aligned with human values. The future 
of AI promises even greater capabilities, but also requires careful consideration 
of potential risks and societal impacts.
"""

if summarization_loaded:
    print("\n" + "=" * 60)
    print("EXAMPLE: Text Summarization")
    print("=" * 60)
    summarize_text(sample_text, max_length=60, min_length=30)

In [ ]:
# Load Question Answering model
print("\n" + "=" * 60)
print("Loading Question Answering Model")
print("=" * 60)

try:
    qa_model = load_model_with_status("deepset/distilbert-base-uncased-squad", "question_answering")
    qa_loaded = True
except:
    qa_loaded = False
    print("⚠ Question Answering model not available")

def answer_question(context, question):
    """
    Answer questions based on provided context
    
    Parameters:
    - context: Reference text
    - question: Question to answer
    """
    if not qa_loaded:
        print("⚠ QA model not loaded")
        return None
    
    print(f"\n❓ Question: {question}")
    
    try:
        result = qa_model(question=question, context=context)
        
        print(f"Answer: {result['answer']}")
        print(f"Confidence: {result['score']:.2%}")
        
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example context and questions
qa_context = """
Python is a high-level, interpreted programming language known for its simplicity 
and readability. Created by Guido van Rossum and first released in 1991, Python has 
become one of the most popular programming languages in the world. It supports multiple 
programming paradigms including procedural, object-oriented, and functional programming. 
Python is widely used in web development, data analysis, artificial intelligence, 
scientific computing, and automation. The language emphasizes code readability with 
its use of significant indentation and clean syntax. The Python Package Index (PyPI) 
provides access to hundreds of thousands of libraries that extend Python's functionality.
"""

if qa_loaded:
    print("\n" + "=" * 60)
    print("EXAMPLE: Question Answering")
    print("=" * 60)
    
    questions = [
        "When was Python created?",
        "What programming paradigms does Python support?",
        "What is the Python Package Index?"
    ]
    
    for q in questions:
        answer_question(qa_context, q)
        print()

## 7. Comparison & Performance Analysis

Evaluate and compare different models on the same tasks.

In [ ]:
import pandas as pd

# Performance comparison function
def benchmark_text_generation(prompt, models_dict, max_length=80):
    """
    Benchmark different text generation models on the same prompt
    
    Parameters:
    - prompt: Input prompt
    - models_dict: Dictionary of model names and loaded models
    - max_length: Generation length
    """
    results = []
    
    print(f"\n📊 Benchmarking {len(models_dict)} models...")
    print(f"Prompt: '{prompt}'\n")
    
    for model_name, model in models_dict.items():
        if model is None:
            print(f"⊘ {model_name}: Not loaded")
            continue
        
        try:
            start_time = time.time()
            output = model(
                prompt,
                max_length=max_length,
                temperature=0.7,
                num_return_sequences=1,
                do_sample=True
            )
            elapsed = time.time() - start_time
            
            generated = output[0]['generated_text']
            
            results.append({
                'Model': model_name,
                'Time (s)': f"{elapsed:.3f}",
                'Output Length': len(generated.split()),
                'Status': '✓'
            })
            
            print(f"✓ {model_name}")
            print(f"  Time: {elapsed:.3f}s | Output length: {len(generated.split())} words")
            
        except Exception as e:
            results.append({
                'Model': model_name,
                'Time (s)': 'Error',
                'Output Length': 'N/A',
                'Status': '✗'
            })
            print(f"✗ {model_name}: {str(e)[:50]}...")
    
    # Display results as table
    if results:
        df = pd.DataFrame(results)
        print("\n" + "=" * 60)
        print("BENCHMARK RESULTS")
        print("=" * 60)
        print(df.to_string(index=False))
    
    return results

# Run benchmark
models_to_compare = {
    "DistilGPT-2": gpt2_model,
    "CodeGen": code_gen_model if code_gen_loaded else None,
}

benchmark_results = benchmark_text_generation(
    "The key to success in machine learning is",
    models_to_compare,
    max_length=80
)

# System information summary
print("\n" + "=" * 60)
print("SYSTEM INFORMATION")
print("=" * 60)
print(f"Device: {device.upper()}")
print(f"PyTorch Version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA Available: Yes")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print(f"CUDA Available: No (running on CPU)")

print("\n" + "=" * 60)
print("KEY TAKEAWAYS")
print("=" * 60)
print("✓ Different models have different strengths:")
print("  - Smaller models (DistilGPT-2): Fast, low memory")
print("  - Larger models (GPT-2, CodeGen): Higher quality, more resources")
print("  - Specialized models: Better for specific tasks")
print("\n✓ Parameters affect output:")
print("  - Temperature: Controls creativity (0=deterministic, 1=random)")
print("  - Top-p: Nucleus sampling for diverse outputs")
print("  - max_length: Balance between quality and speed")
print("\n✓ Tips for production use:")
print("  - Use quantization for smaller model sizes")
print("  - Implement caching to avoid reloading models")
print("  - Consider API services for larger models")
print("  - Monitor GPU memory to avoid out-of-memory errors")

## 8. Next Steps & Resources

Expand your generative AI skills and applications.

### Recommended Enhancements:

1. **Hugging Face Hub Integration**
   - Explore thousands of pre-trained models at huggingface.co
   - Fine-tune models on your custom data
   - Use `huggingface_hub` library for model management

2. **Ollama for Local LLMs**
   - Run large language models locally without internet
   - Models: Llama 2, Mistral, Neural Chat, etc.
   - Visit ollama.ai to get started

3. **API Integration**
   - OpenAI API for GPT-4 and ChatGPT
   - Google Gemini API
   - Anthropic Claude API

4. **Advanced Techniques**
   - Prompt engineering and optimization
   - Few-shot learning
   - Retrieval-Augmented Generation (RAG)
   - Model quantization for efficiency
   - LoRA fine-tuning for domain adaptation

5. **Deployment Options**
   - Streamlit for web interfaces
   - FastAPI for REST APIs
   - Gradio for quick demos
   - Docker containerization
   - Cloud platforms (AWS, Google Cloud, Azure)

### Useful Libraries:
```python
# Text generation and NLP
pip install transformers torch diffusers accelerate

# API clients
pip install openai anthropic google-generativeai

# Web frameworks
pip install streamlit fastapi gradio

# Model optimization
pip install optimum bitsandbytes

# Data science
pip install pandas scikit-learn matplotlib seaborn
```

### Learning Resources:
- **Hugging Face Course**: huggingface.co/course
- **Papers with Code**: paperswithcode.com
- **Model Cards**: huggingface.co/models
- **Prompt Engineering Guide**: github.com/dair-ai/Prompt-Engineering-Guide